In [ ]:
import os
import geopandas as gpd
import pandas as pd 
import logging
import numpy as np 
from pathlib import Path
from shapely.geometry import Point, LineString, MultiLineString
from shapely.ops import linemerge
from shapely import wkt # for WKT 轉幾何物件
from TDXdataframe import read_businfo_xml
from basicprocess import create_folder, findfiles, read_combined_dataframe, get_df_log, outputlog

# 函數庫

## Trash

In [ ]:
# # ===== 後續不會用到 =====
# def get_gdfroutepair(df_seq):
#     # 先排序，確保順序正確
#     df_seq = df_seq.sort_values(
#         by=['RouteUID', 'SubRouteUID', 'Direction', 'StopSequence']
#     )

#     # 建立 To 欄位（往下一站）
#     df_seq['ToStop'] = df_seq.groupby(
#         ['RouteUID', 'SubRouteUID', 'Direction']
#     )['StopUID'].shift(-1)

#     df_seq['ToSeq'] = df_seq.groupby(
#         ['RouteUID', 'SubRouteUID', 'Direction']
#     )['StopSequence'].shift(-1)

#     df_seq['ToLon'] = df_seq.groupby(
#         ['RouteUID', 'SubRouteUID', 'Direction']
#     )['PositionLon'].shift(-1)

#     df_seq['ToLat'] = df_seq.groupby(
#         ['RouteUID', 'SubRouteUID', 'Direction']
#     )['PositionLat'].shift(-1)

#     # 建立 From 欄位（本站）
#     df_seq['FromSeq'] = df_seq['StopSequence']
#     df_seq['FromLon'] = df_seq['PositionLon']
#     df_seq['FromLat'] = df_seq['PositionLat']
#     df_seq['FromStop'] = df_seq['StopUID']

#     # 只保留需要的欄位
#     df_result = df_seq.reindex(columns = [
#         'RouteUID', 'SubRouteUID', 'Direction',
#         'FromStop', 'FromSeq', 'FromLon', 'FromLat',
#         'ToStop', 'ToSeq', 'ToLon', 'ToLat'])

#     # 移除最後一站（沒有 To）
#     df_result = df_result.dropna(subset=['ToSeq'])

#     df_result = df_result.reset_index(drop=True)

#     gdf_result = get_line(df_result, x1 = 'FromLon', x2 = 'ToLon', y1 = 'FromLat', y2 = 'ToLat')

#     return gdf_result


## Used

In [ ]:
# 01_geodataframe 圖像處理
def dataframe_to_point(df, lon_col, lat_col, crs="EPSG:4326", target_crs="EPSG:3826"):
    '''
    Parameters:
    df (dataframe) : 含經緯度座標欄位的dataframe
    lon_col (str) : 緯度欄位
    Lat_col (str) : 經度欄位
    crs (str) : 目前經緯度座標的座標系統，常用的為4326(WGS84)、3826(TWD97)
    target_crs：目標轉換的座標系統
    '''

    # from shapely.geometry import Point
    # import pandas as pd
    # import geopandas as gpd
    # Create Point geometries from the longitude and latitude columns
    geometry = [Point(xy) for xy in zip(df[lon_col], df[lat_col])]
    # Create a GeoDataFrame with the original CRS
    gdf = gpd.GeoDataFrame(df, geometry=geometry, crs=crs)
    # Convert the GeoDataFrame to the target CRS
    gdf = gdf.to_crs(epsg=target_crs.split(":")[1])
    return gdf

def get_line(df, x1 = 'Lon_o', x2 = 'Lon_d', y1 = 'Lat_o', y2 = 'Lat_d', target_crs = "EPSG:3826"):
    '''
    Parameters:
    df (dataframe) : 含經緯度座標欄位的dataframe
    x1 (str) : 起點經度欄位
    y1 (str) : 起點緯度欄位
    x2 (str) : 迄點經度欄位
    y2 (str) : 迄點緯度欄位

    預設立場：輸出為wgs84轉換的經緯度點位
    '''
    # from shapely.geometry import LineString
    # import pandas as pd
    # import geopandas as gpd
    df['geometry'] = df.apply(lambda row: LineString([(row[x1], row[y1]), (row[x2], row[y2])]), axis=1)
    gdf = gpd.GeoDataFrame(df, geometry='geometry')
    # 設定座標系統 (假設 WGS 84 / EPSG:4326)
    gdf.set_crs(epsg=4326, inplace=True)
    gdf = gdf.to_crs(epsg=target_crs.split(":")[1])
    return gdf

def get_line_endpoints(geom):
    if geom is None or geom.is_empty:
        return (None, None)

    # 如果是 MultiLineString，先 merge（能接成一條就會變 LineString）
    if isinstance(geom, MultiLineString):
        geom = linemerge(geom)

        # merge 後可能還是 MultiLineString（不連續），那就選最長那段
        if isinstance(geom, MultiLineString):
            geom = max(list(geom.geoms), key=lambda g: g.length)

    # 如果是 LineString
    if isinstance(geom, LineString):
        return (geom.coords[0], geom.coords[-1])

    # 其他幾何型別（保險）
    return (None, None)

def angle_math(from_pt, end_pt):
    if from_pt is None or end_pt is None:
        return np.nan
    x1, y1 = from_pt
    x2, y2 = end_pt
    dx, dy = x2 - x1, y2 - y1
    ang = np.degrees(np.arctan2(dy, dx))  # -180~180
    return (ang + 360) % 360              # 0~360

def get_move_dir(angle, spdir):
    if angle is None or np.isnan(angle) or spdir is None:
        return None

    angle = angle % 360

    # 南北向
    if spdir == '往南/往北':
        if angle < 180:
            return '北'
        else:
            return '南'

    # 東西向
    if spdir == '往東/往西':
        if (angle >= 315) or (angle < 135):
            return '東'
        else:
            return '西'

    return None




In [ ]:
def get_screenline_and_surveypoint(SLfolder):
    gdf_sl = gpd.read_file(os.path.join(SLfolder, 'TRTS5_屏柵線線型.shp'))
    gdf_sl['Category'] = gdf_sl['TRTS5'].apply(
        lambda x: 0 if 'CD' in x else 1 if 'SL' in x else None
    )
    gdf_sl['Number'] = (
        gdf_sl['TRTS5']
        .str.extract(r'(\d+)')
        .astype(int)
    )
    gdf_sl = gdf_sl.sort_values(['Category', 'Number'])

    gdf_sp = gpd.read_file(os.path.join(SLfolder, 'TRTS5_全部調查點位.shp'))
    gdf_sp = gdf_sp.rename(columns = {'X':'PositionLon', 'Y':'PositionLat'})
    gdf_sp[['TRTS5', 'SP_No']] = gdf_sp['Name'].str.split('-', expand=True)
    gdf_sp['SP_No'] = gdf_sp['SP_No'].astype('int64')
    gdf_sp['Category'] = gdf_sp['TRTS5'].apply(
        lambda x: 0 if 'CD' in x else 1 if 'SL' in x else None
    )
    gdf_sp['Number'] = (
        gdf_sp['TRTS5']
        .str.extract(r'(\d+)')
        .astype(int)
    )
    gdf_sp = gdf_sp.sort_values(['Category', 'Number', 'SP_No'])

    return gdf_sl, gdf_sp

def get_gdf_sp_unique(gdf_sp):
    '''因為調查點位同一個名稱的有太多資料調查來源：先取平均'''
    df_sp_unique = gdf_sp.sort_values(['Category','TRTS5', 'SP_No']).groupby(['Category','TRTS5', 'SP_No']).agg({'Name':'first','PositionLon':'mean', 'PositionLat':'mean', 'Surveyname':'first'}).reset_index()
    gdf_sp_unique = dataframe_to_point(df = df_sp_unique, lon_col = 'PositionLon', lat_col = 'PositionLat', crs="EPSG:4326", target_crs="EPSG:3826")
    return gdf_sp_unique

def get_gdf_sp_manual(excelpath,
                      sheet_name="調查點位"):
    df = pd.read_excel(excelpath , sheet_name="調查點位")
    df.drop(columns = 'geometry', inplace = True)
    gdf = dataframe_to_point(df, 'PositionLon', 'PositionLat', crs="EPSG:4326", target_crs="EPSG:3826")

    return gdf

def get_df_seq(seqfolder):
    '''讀取公車站序並轉為geodataframe
    seqfolder(str):站序點位資料csv所在資料夾'''
    df_seq = read_combined_dataframe(file_list=findfiles(seqfolder))
    df_seq = df_seq.reindex(columns = ['RouteUID', 'SubRouteUID', 'SubRouteName_Zh','Direction','StopUID', 'StopSequence', 'PositionLon', 'PositionLat']).rename(columns = {'SubRouteName_Zh':'SRouteName'})
    df_seq = df_seq.drop_duplicates(subset= ['RouteUID', 'SubRouteUID','Direction', 'StopSequence', 'PositionLon', 'PositionLat'])
    if (len(df_seq[df_seq['StopSequence'].isna()]) / len(df_seq) < 0.05):
        df_seq = df_seq[df_seq['StopSequence'].notna()] # 會有沒有站序的
    else:
        percentage = ((df_seq[df_seq['StopSequence'].isna()]) / len(df_seq)) * 100
        print(f"===== 需要檢查沒有站序的站點有哪些，共有{percentage}%的資料沒有站序 =====")

    return df_seq

def buffer_distance(gdf, buffermeters):
    gdf = gdf.copy()
    gdf['geometry'] = gdf.geometry.buffer(buffermeters)
    return gdf 

def get_gdf_route(routefolder):
    df_route = read_combined_dataframe(findfiles(routefolder))
    df_route["geometry"] = df_route["Geometry"].apply(wkt.loads)
    df_route = df_route.drop(columns = ['Geometry'])
    gdf_route = gpd.GeoDataFrame(
        df_route,
        geometry="geometry",
        crs="EPSG:4326"
    ) 

    return gdf_route

def get_route_direction(route_in_buffer, 
                        pointdir_col = 'SPDir',
                        output_col ='MoveDir'):
    route_in_buffer = route_in_buffer.copy()

    logging.info(f'取出交集後公車片段的起訖點')
    route_in_buffer[['FromPoint', 'EndPoint']] = route_in_buffer.geometry.apply(
        lambda g: get_line_endpoints(g)
        ).apply(pd.Series)

    logging.info(f'計算公車行進角度')
    route_in_buffer['Angle360'] = route_in_buffer.apply(
        lambda r: angle_math(r['FromPoint'], r['EndPoint']),
        axis=1
    )

    route_in_buffer[output_col] = route_in_buffer.apply(
        lambda r: get_move_dir(r['Angle360'], r[pointdir_col]),
        axis=1
    )

    route_in_buffer = route_in_buffer.drop(columns = ['FromPoint', 'EndPoint', 'Angle360'])

    return route_in_buffer

def getid(df, collist, outputcol, sep='-'):
    """
    將多個欄位轉成字串後串接，產生一個新欄位

    Parameters
    ----------
    df : pandas.DataFrame
        原始資料表
    collist : list
        要串接的欄位名稱清單
    outputcol : str
        輸出的欄位名稱
    sep : str, optional
        欄位之間的分隔符號，預設為 '-'
    """
    df[outputcol] = (
        df[collist]
        .fillna('')
        .astype(str)
        .agg(sep.join, axis=1)
    )
    return df

def remove_duplicate_route(route_in_buffer):
    route_in_buffer = route_in_buffer.copy()
    route_in_buffer = getid(route_in_buffer,
                            collist = ['RouteUID', 'SRouteUID', 'Direction', 'Name'],
                            outputcol = 'CHECKUID',
                            sep='_')
    dup_checkid = route_in_buffer[route_in_buffer.duplicated(subset=['CHECKUID'], keep=False)]
    route_in_buffer_clean = route_in_buffer[ ~route_in_buffer.duplicated(subset=['CHECKUID'], keep=False)]
    dup_checkid = dup_checkid.sort_values(['CHECKUID', 'Len_Per', 'SelectLen'], ascending=[True, True, True]).drop_duplicates(subset='CHECKUID')

    clean_route_in_buffer = pd.concat([route_in_buffer_clean, dup_checkid])
    clean_route_in_buffer.sort_index()

    clean_route_in_buffer = clean_route_in_buffer.drop(columns = ['CHECKUID'])

    return clean_route_in_buffer


# Setup

In [ ]:
# 00_Setup 所有全域函數
# 0.) Log輸出檔案位置
logfile = os.path.abspath(os.path.join(os.getcwd(), '..', 'Log', '03_SelectSegment.log'))
if os.path.exists(logfile):
    os.remove(logfile)
    
logging.basicConfig(
    filename=logfile,
    level=logging.INFO,
    format="%(asctime)s | %(levelname)s | %(message)s"
)


# 1.) 參數
SL_buffer_meters = 500
SP_buffer_meters = 50

# 2.) 資料input資料夾
referencefolder = os.path.abspath(os.path.join(os.getcwd(), '..', '參考資料'))
seqfolder = os.path.abspath(os.path.join(os.getcwd(), '..', '00_TDX資料下載', '01公車站序資料'))
routefolder = os.path.abspath(os.path.join(os.getcwd(), '..', '00_TDX資料下載', '02公車路線資料'))
SLfolder = os.path.abspath(os.path.join(os.getcwd(), '..', '..', 'TRTS5屏柵線'))
routesegmentfolder = os.path.abspath(os.path.join(os.getcwd(), '..', '03_處理後資料', '01_公車路線依站序拆分'))
routeselectsegmentoutputfolder = create_folder(os.path.abspath(os.path.join(os.getcwd(), '..', '03_處理後資料', '02_調查點位公車路段')))


# Main

In [ ]:
def main():

    # 1. 讀取資料
    logging.info('Job started')
    logging.info('Start reading Data')
    # 讀取模型編修組提供的屏柵線 & 調查點位
    gdf_sl, gdf_sp = get_screenline_and_surveypoint(SLfolder=SLfolder)
    logging.info(f'讀取屏柵線檔案 該線形crs為 {gdf_sl.crs.name}')
    logging.info(f'讀取調查點檔案 該點位crs為 {gdf_sp.crs.name}')

    # 轉為TWD97 (EPSG:3826) 才能使用buffer功能
    # gdf_sp_unique = get_gdf_sp_unique(gdf_sp).to_crs(epsg=3826) # 因為提供的點位有分路段，所以先改為同一個地點
    gdf_sp_unique = get_gdf_sp_manual(excelpath = os.path.join(referencefolder, '屏柵線.xlsx'),  sheet_name="get_gdf_sp_manual") # 這邊已經進行人工挑選點位，並決定後續挑選方向的方式
    logging.info(f'讀取調查點檔案 該點位crs為 {gdf_sp_unique.crs.name}')

    gdf_sl_twd97 = gdf_sl.to_crs(epsg=3826).reindex(columns = ['TRTS5', 'Category', 'Number', 'geometry']) # 轉換為TWD97處理
    logging.info(f'讀取屏柵線檔案 該線形crs為 {gdf_sl_twd97.crs.name}')

    # 讀取公車站序
    df_seq = get_df_seq(seqfolder=seqfolder)
    gdf_seq = dataframe_to_point(df = df_seq, lon_col='PositionLon', lat_col= 'PositionLat', crs="EPSG:4326", target_crs="EPSG:3826")
    logging.info(f'讀取公車站序geodataframe 該點位crs為 {gdf_seq.crs.name}')

    # 讀取公車原始線型
    gdf_route  = get_gdf_route(routefolder)
    gdf_route = gdf_route.to_crs(epsg=3826)
    logging.info(f'讀取公車原始線形 該點位crs為 {gdf_route.crs.name}')

    # 讀取公車片段
    gdf_route_segment = gpd.read_file(os.path.join(routesegmentfolder, 'gdf_segment_routes_TWD97.shp'))
    logging.info(f'讀取公車路線拆分片段 該點位crs為 {gdf_route_segment.crs.name}')


    # 2. Buffer初步確認那些會經過調查點位
    # gdf_sl_buffer = buffer_distance(gdf=gdf_sl_twd97, buffermeters=SL_buffer_meters).reset_index(drop = True) # 屏柵線取buffer
    gdf_sp_buffer = buffer_distance(gdf=gdf_sp_unique, buffermeters=SP_buffer_meters).reset_index(drop = True) # 調查點位取buffer
    logging.info(f'調查點位進行環域，還域範圍為 {SP_buffer_meters}')

    logging.info('以調查點位環域與公車路線片段進行空間交集')
    # route_in_buffer = gpd.clip(gdf_route_segment,gdf_sp_buffer)
    route_in_buffer = gpd.overlay(
                        gdf_route_segment,
                        gdf_sp_buffer,
                        how="intersection")
    route_in_buffer['SelectLen'] = route_in_buffer.geometry.length
    logging.info(f'原本輸入的公車片段共有 {len(gdf_route_segment):,} 筆資料')
    logging.info(f'實際取得交集的公車片段共有 {len(route_in_buffer):,} 筆資料')


    # 3. 判斷公車方向
    logging.info(f'開始判斷公車方向')
    route_in_buffer = get_route_direction(route_in_buffer)
    
    
    # 4. 檢核同一個路段是否有重複
    route_in_buffer = remove_duplicate_route(route_in_buffer)
    logging.info(f'成功刪除重複路段後，剩下 {len(route_in_buffer):,} 筆資料')
    route_in_buffer.to_csv(os.path.join(routeselectsegmentoutputfolder,'公車路線片段選取結果.csv'), index=False, encoding='utf-8-sig')
    logging.info(f'輸出至 {os.path.join(routeselectsegmentoutputfolder,"公車路線片段選取結果.csv")}')

    logging.info(f'Job ended')
    outputlog(logfile=logfile)

    return route_in_buffer


if __name__ == "__main__":
    route_in_buffer = main()

